In [1]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [2]:
#lookup = 'BMW Landshut 4.10'
#lookup = 'GRR'
#lookup = 'V441'
lookup = 'Forsa Hartmoor'
#lookup = 'OSPEDALE DI BORGO TRENTO'
#lookup = 'FORD SAARLOUIS'
#lookup = 'KRONOSPAN'
motor_num = 1

In [3]:
def sfun(x):
    #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')

In [4]:
fleet.T

,0,1,2,3,4,5,6,7,8,9,10
Count_OpHour,281.0,282.0,270.0,263.0,277.0,283.0,268.0,253.0,255.0,218.0,253.0
OperationalCondition,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available
startup_counter,201.0,161.0,152.0,152.0,152.0,153.0,144.0,138.0,147.0,134.0,142.0
shutdown_counter,201.0,163.0,152.0,152.0,153.0,155.0,144.0,138.0,147.0,136.0,147.0
id,159396,159397,159399,159398,159400,159401,159403,159402,159404,159405,159406
Engine Version,H12,H12,H12,H12,H12,H12,H12,H12,H12,H12,H12
Engine Type,624,624,624,624,624,624,624,624,624,624,624
Engine Series,6,6,6,6,6,6,6,6,6,6,6
Country,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB
Commissioning Date,2022-01-10,2022-01-10,2022-01-11,2022-01-11,2022-01-22,2022-01-10,2022-01-11,2022-01-11,2022-01-11,2022-01-22,2022-01-11


In [5]:
motor = fleet.iloc[motor_num]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=Engine.from_fleet(mp,motor)
pp_from=e['Commissioning Date']
#pp_from='2020-01-01'
#pp_from='2022-04-01 06:00'
#pp_from='2022-03-28 06:00'
#pp_to='2022-03-28 08:14'
#pp_to='2022-04-12'
pp_to=datetime.now()
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('Forsa Hartmoor M02',
 ['undefined', 'OFF', 'MAN', 'AUTO'],
 ['success', 'failed', 'undefined'])

In [6]:
# Run State Machines
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=True, debug=False)
fsm.run1(silent=True, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent = True, debug = True)
fsm.store()

In [7]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
#rda[startstopFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")
pd.DataFrame.from_dict(e.dash, orient='index').T

Starts: 188, Successful: 0, Failed: 0 => 0.0%


,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,Forsa Hartmoor M02,M02,AL87,624,H12,24,4495.0,24.5,1486152,159397,304.0,2022-01-10,54,250.0,0.12


In [8]:
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success']) & (rda.targetload > 2800.0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda.count_alarms > 0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde = rda
rde['datetime'] = pd.to_datetime(rde['starttime'])
rde['isuccess'] = rde.apply(lambda x: 1 if x['success'] else 0, axis=1)
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
display(rde[vec].describe().round(2))
dfigsize = (22,10)
dset = [
    {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
    {'col':['runout'],'_ylim':(0,100) },
    {'col':['targetload'],'_ylim':(-4000,26000) },
    {'col':['ramprate'],'_ylim':(-5,7)},
    {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
    {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
    {'col':['synchronize'],'_ylim':(-20,400)},
    {'col':['startpreparation'],'_ylim':(-1000,800)},
    {'col':['count_warnings','count_alarms','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
    {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
    #{'col':['count_warnings','count_alarms','no'],'ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,0,0,0.1)'] }
    ]

dset = equal_adjust(dset, rde, do_not_adjust=[-1], qmin=0.1)
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)

,startpreparation,speedup,idle,synchronize,loadramp,targetload,ramprate,cumstarttime,targetoperation,rampdown,coolrun,runout,isuccess
count,188.00,146.00,140.00,139.00,139.00,135.00,135.00,188.00,125.00,139.00,136.00,135.00,188.0
mean,91.26,20.62,5.10,34.81,112.16,4372.73,0.91,222.85,6527.00,119.75,10.24,49.95,1.0
std,65.25,2.92,3.17,25.10,39.45,640.12,0.29,113.44,6026.29,24.34,2.77,1.79,0.0
min,0.60,8.38,4.03,0.80,0.30,1192.54,0.48,2.43,29.72,0.30,9.98,39.63,1.0
25%,61.41,18.87,4.34,6.26,86.08,4493.20,0.70,180.36,1821.04,126.90,9.99,48.50,1.0
50%,100.36,20.29,4.54,38.13,127.26,4517.99,0.78,278.17,4870.60,126.99,10.00,50.95,1.0
75%,102.33,21.98,5.14,58.21,142.54,4539.80,1.12,279.75,10036.30,127.09,10.01,51.46,1.0
max,450.28,42.56,39.35,81.78,210.09,4642.34,2.38,532.55,37489.88,127.35,42.30,52.44,1.0


In [9]:
out = widgets.Output()
pfigsize=(20,12)

def myfigures(e):
    return {
    'actors' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
        ],
        'tecjet' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        ],
        'lubrication' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 3], 'color':'black', 'unit': 'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
        {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
        ],
        'exhaust' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
        {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 12000], 'unit':'mV'},
        ],
        'ignition' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
        {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
        {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
        ],    
    }

def update_fig(x=0):
    global vv; 
    rdbs = rdb[rdb.no == x]
    if not rdbs.empty:
        startversuch = rdbs.iloc[0]
        #startversuch = rdb.iloc[x]
        vv = startversuch.no; 
        #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
        #display(HTML(ftitle));
        #disp_result(startversuch)

        # PLotter
        lfigures = myfigures(fsm._e)
        plotdef, vset = cplotdef(mp, lfigures)
        #dmaxlength = 1800
        dmaxlength = None
        dminlength = None
        #startversuch = rdb.iloc[vv]
        ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig_handles = []
        for doplot in plotdef:
            dset = lfigures[doplot]
            ltitle = f"{ftitle} | {doplot}"
            if count_columns(dset) > 12: # no legend, if too many lines.
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=pfigsize)
            else:
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=pfigsize)

            fig = FSM_add_Notations(fig, fsm, startversuch)
            disp_alarms(startversuch)
            disp_warnings(startversuch)
            fig = FSM_add_Alarms(fig, fsm, startversuch)
            fig = FSM_add_Warnings(fig, fsm, startversuch)
            fig_handles.append(bokeh_show(fig, notebook_handle=True))
        for h in fig_handles:
            push_notebook(handle=h)
    else:
        print(f"Start No {x} is not in the filtered results. please consider changing filters, if you want to access it.")

In [11]:
vv = 0
@out.capture(clear_output=True)
def show_plots(but):
    update_fig(sno.value)

def html_summary(*args):
    if not rdf.empty:
        sv = rdf.iloc[sno.value]
        global vv; vv = sno.value
        s = '''
        <style>
            table, td {
                border: 1px solid grey;
                border-collapse: collapse;
                padding: 0px 5px;
                text-align: right;
            }
            thead {
                text-align: left;
            }
        </style>
        '''
        summary = pd.DataFrame(sv[startstopFSM.run2filter_content]).T
        #display(summary)
        r = f"{s}{summary.to_html(escape=False, index=False)}"
        start_table.value = r
    
vv = 0 
sno = widgets.IntText(
    description='StartNo: ',
    layout=widgets.Layout(width='236px'))
sno.observe(html_summary, 'value')
sno_slider = widgets.IntSlider(0, 0, max(rdf.shape[0]-1,1) , 1,
    description = 'StartNo:',
    layout=widgets.Layout(width='516px'))
mylink = widgets.jslink((sno, 'value'), (sno_slider, 'value'))

tshowplots = widgets.Button(description='Show Plots',disabled=False, button_style='primary')
tshowplots.on_click(show_plots)
start_table = widgets.HTML()
html_summary()

display( widgets.VBox([widgets.HBox([sno_slider, sno, tshowplots]),start_table, out]));

In [ ]:
startversuch = rdf.iloc[vv]
startversuch

In [ ]:
startversuch = rdf.iloc[vv]
print("messages leading to state Switches:")
print("-----------------------------------")
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")
print(f"\nall messages during start attempt No.:{vv:4d} leading to state Switches:")
print("---------------------------------------------------------------------")
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
rdb = rda
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(_=rdb[vec].hist(bins=30,layout=(4,4),figsize=(20,20)))
#ax_list[0][2].set_xlim((0,10))
#display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))

In [ ]:
import time
import dmyplant2
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet

dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
t0 = time.time()
for i, startversuch in rdb[-2:-1].iterrows() : 
#for i, startversuch in tqdm(rdb.iterrows(), total=rdb.shape[0], ncols=80, mininterval=1, unit=' starts', desc="FSM Run2"):
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(14,6))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        #fig = FSM_add_Alarms(fig, fsm, startversuch)
        #fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")
t1 = time.time()
print(f"{(t1-t0):4.1f} sec")    

In [ ]:
mfn = e._fname + '_messages.txt'
#fsm.save_messages(mfn)
print(mfn)